In [29]:
import json

# Open and read the JSON file
with open('json_data/transcripts_2024.json', 'r') as file:
    data = json.load(file)

In [30]:
new_data_ls = []

for sub_ls in data[100:]:
    for i in sub_ls:
        new_data_ls.append(i)

In [32]:
len(new_data_ls)

5861

In [64]:
import pandas as pd

df=pd.DataFrame(new_data_ls)

In [65]:
st_ls = ['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA', 'CRSP', 'ABNB', 'ACN', 'ADYEY', 'ASML', 'CRM', 'EUXTF', 'FSLR', 'IBDSF', 'LDNXF', 'META', 'NVDA', 'SBGSF', 'SHOP', 'SPGI', 'TOITF', 'TTD', 'TTE', 'V'\
    'PATH', 'GCT', 'TMDX', 'NET', 'AXON', 'PL', 'OKTA', 'APP', 'CRWD', 'LULU', 'RKLB', 'NKE', 'AVGO', 'ZETA', 'UBER', 'SCMI', 'ISRG', 'TSM', 'ADSK', 'ARM', 'DDOG', 'HESAF', 'MDB', 'NOW', 'LVMHF', 'TWLO', 'ZTS']

In [67]:
subset = df[df['symbol'].isin(st_ls)].drop_duplicates(subset=['symbol', 'date']).to_dict('records')

In [63]:
for i in new_data_ls:
    file_name = 'data/'+ str(i['year']) + '_' + str(i['quarter']) + '_' + i['symbol']+'.txt'
    with open(file_name, 'w') as file:
        file.write(i['content'])

In [4]:
with open('transcripts_2024.json', 'w') as f:
    json.dump(new_data_ls, f)

In [8]:
from llama_index.readers.json import JSONReader

# Initialize JSONReader
reader = JSONReader(
    # The number of levels to go back in the JSON tree. Set to 0 to traverse all levels. Default is None.
    levels_back=0,
    # The maximum number of characters a JSON fragment would be collapsed in the output. Default is None.
    collapse_length=None,
    # If True, ensures that the output is ASCII-encoded. Default is False.
    ensure_ascii=True,
    # If True, indicates that the file is in JSONL (JSON Lines) format. Default is False.
    is_jsonl=True,
    # If True, removes lines containing only formatting from the output. Default is True.
    clean_json=True,
)

# Load data from JSON file
documents = reader.load_data(input_file="transcripts_2024.json", extra_info={})
documents = documents[:6]
#docs to nodesj

In [4]:
from transcripts_rag.local_llama import load_model

llm = load_model()

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  

In [2]:
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings

# llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
# embed_model = CohereEmbeddings(model="embed-english-v3.0")

In [5]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.json import JSONReader
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.extractors.entity import EntityExtractor

transformations = [
    #JSONReader(levels_back=0, collapse_length=None,ensure_ascii=True,  is_jsonl=True,clean_json=True ),
    SentenceSplitter(),
    TitleExtractor(nodes=5, llm=llm),
    #QuestionsAnsweredExtractor(questions=3, llm=llm),
    #SummaryExtractor(summaries=["prev", "self"], llm=llm),
    KeywordExtractor(keywords=5, llm=llm),
    EntityExtractor(prediction_threshold=0.5),
]

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
import json

import json

# Open and read the JSON file
with open('transcripts_2024.json', 'r') as file:
    documents = json.load(file)
documents = documents[:6]

In [9]:
from llama_index.core.ingestion import IngestionPipeline


async def run_pipeline(documents):
    pipeline = IngestionPipeline(transformations=transformations)
    nodes = await pipeline.arun(documents=documents)
    return nodes

In [10]:
import asyncio

import nest_asyncio
nest_asyncio.apply()

asyncio.run(run_pipeline(documents))


  0%|          | 0/5 [00:00<?, ?it/s]
llama_print_timings:        load time =    6124.42 ms
llama_print_timings:      sample time =      14.90 ms /   256 runs   (    0.06 ms per token, 17182.36 tokens per second)
llama_print_timings: prompt eval time =    8578.71 ms /  1100 tokens (    7.80 ms per token,   128.22 tokens per second)
llama_print_timings:        eval time =    9341.07 ms /   255 runs   (   36.63 ms per token,    27.30 tokens per second)
llama_print_timings:       total time =   18134.93 ms /  1355 tokens
Llama.generate: 62 prefix-match hit, remaining 1010 prompt tokens to eval

llama_print_timings:        load time =    6124.42 ms
llama_print_timings:      sample time =      15.88 ms /   256 runs   (    0.06 ms per token, 16116.85 tokens per second)
llama_print_timings: prompt eval time =    4049.28 ms /  1010 tokens (    4.01 ms per token,   249.43 tokens per second)
llama_print_timings:        eval time =    9441.77 ms /   255 runs   (   37.03 ms per token,    27.01 tok

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/michieldekoninck/nltk_data'
    - '/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/nltk_data'
    - '/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/share/nltk_data'
    - '/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/transcripts_rag/lib/python3.10/site-packages/llama_index/core/_static/nltk_cache'
*******************************************

In [ ]:

from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)

class CustomExtractor(BaseExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": (
                    node.metadata["document_title"]
                    + "\n"
                    + node.metadata["excerpt_keywords"]
                )
            }
            for node in nodes
        ]
        return metadata_list

In [23]:
documents[0].text.index('content')

77

In [27]:
documents[0].text[:76]

'"symbol": "DNB",\n"quarter": 2,\n"year": 2024,\n"date": "2024-08-03 13:51:09",\n'

In [ ]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        TitleExtractor(),
        OpenAIEmbedding(),
    ]
)

# run the pipeline
nodes = pipeline.run(documents=[Document.example()])